# Pre-processing

This pre-processing recieves a .xlsx file and returns a processed.xlsx file. 

In [234]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib  # For model serialization (pickling)

In [237]:
excel_file = pd.ExcelFile('D2lData.xlsx')

# Reading each sheet into a DataFrame
df_d2l = pd.read_excel(excel_file, 'd2l')
df_demo = pd.read_excel(excel_file, 'demographics')
df_grades = pd.read_excel(excel_file, 'grades')

In [236]:
def df_construct(df_d2l, df_demo, df_grades):
    
    # Drop duplicates based on the specified columns, keeping the first occurrence (which has the largest 'content_completed')
    df_d2l.sort_values(by='content_completed', ascending=False, inplace=True)
    df_d2l.drop_duplicates(subset=['pseudo_id', 'pseudo_course', 'term'], keep='first', inplace=True)
    
    # Drop all date columns
    df_d2l.drop(columns=['last_discussion_post_date', 'last_assignment_submission_date', 'last_system_login',
                         'last_quiz_attempt_date', 'last_visited_date'], axis=1, inplace=True)
    
    # appending the "grade_value" column into the df_d2l dataframe (if there is no grade, a zero is input)
    df_d2l['grade_value'] = df_d2l.apply(lambda row: df_grades[
        (df_grades['pseudo_student_id'] == row['pseudo_id']) & 
        (df_grades['pseudo_course_name'] == row['pseudo_course']) &
        (df_grades['term'] == row['term'])
    ]['grade_value'].values[0] if not df_grades[
        (df_grades['pseudo_student_id'] == row['pseudo_id']) & 
        (df_grades['pseudo_course_name'] == row['pseudo_course']) &
        (df_grades['term'] == row['term'])
    ].empty else 0, axis=1)
    
    # append 'gender', 'imm_status', 'age' onto dataframe - joined on student id
    merged_df = pd.merge(df_d2l, df_demo[['pseudo_student_id', 'gender', 'imm_status', 'age']], 
                         left_on='pseudo_id', right_on='pseudo_student_id', how='left')
    merged_df.drop('pseudo_student_id', axis=1, inplace=True)
    
    # Append the total course counts for each student
    course_counts_df = df_grades.groupby('pseudo_student_id')['pseudo_course_name'].count().reset_index()
    course_counts_df.columns = ['pseudo_student_id', 'total_course_count']

    merged_df = pd.merge(merged_df, course_counts_df, left_on='pseudo_id', right_on='pseudo_student_id', how='left')
    merged_df.drop('pseudo_student_id', axis=1, inplace=True)
    
    # Append the total course counts for each student by term
    course_counts_by_term_df = df_grades.groupby(['pseudo_student_id','term'])['pseudo_course_name'].count().reset_index()
    course_counts_by_term_df.columns = ['pseudo_student_id', 'term', 'course_count_by_term']

    merged_df = pd.merge(merged_df, course_counts_by_term_df, left_on=['pseudo_id','term'], right_on=['pseudo_student_id', 'term'], how='left')
    merged_df.drop('pseudo_student_id', axis=1, inplace=True)
    
    # drop student id from df
    merged_df.drop('pseudo_id', axis=1, inplace=True)
    
    # Apply the function to create the 'at_risk' column that classifies any students with a 1.0 or below as "at-risk"
    merged_df['at_risk'] = merged_df['grade_value'].apply(lambda grade: 1 if grade < 1.0 else 0)
    
    # drop grade_value from df
    merged_df.drop('grade_value', axis=1, inplace=True)
    
    # Add engineered values
    merged_df = add_eng_values(merged_df)
    
    # Alter categorical columns as necessary
    merged_df = alter_term_gender(merged_df)
    
    return merged_df

In [238]:
df = df_construct(df_d2l, df_demo, df_grades)

In [240]:
X = df.drop('at_risk', axis=1)
y = df.at_risk

In [179]:
# A function to add the engineered values - completion ratio, logins per course, average time spent per course login

def add_eng_values(X):
    
    X['completion_ratio'] = X['content_completed']/X['content_required']
    X['logins_per_course'] = X['number_of_logins_to_the_system']/X['total_course_count']
    X['avg_time_per_login'] = X['total_time_spent_in_content']/X['logins_per_course']
    
    X['completion_ratio'].fillna(0, inplace=True)
    X['avg_time_per_login'].fillna(0, inplace=True)
    X['course_count_by_term'].fillna(0, inplace=True)
    
    return X

# Define the transformer
eng_values_transformer = FunctionTransformer(add_eng_values, validate=False)

In [175]:
# Create a function for the logarithmic transformation
log_transformer = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

In [196]:
def alter_term_gender(X_cat):
    
    # Alter format of 'term' value from 'DDDDS' to 'S'
    X_cat['term'] = X_cat['term'].str[-1]
    
    # Replace missing values in Gender column with 'U' for "undeclared"
    X_cat['gender'].fillna('U', inplace=True)
    
    return X_cat

# Define the transformer
term_gender_transformer = FunctionTransformer(alter_term_gender, validate=False)

In [247]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [248]:
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Create preprocessing transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('log_transform', log_transformer)
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define your model
model = RandomForestClassifier()

# Create the final pipeline
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

In [249]:
# Fit the pipeline to your training data
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('log_transform',
                                                                   FunctionTransformer(func=<ufunc 'log1p'>,
                                                                                       inverse_func=<ufunc 'expm1'>))]),
                                                  ['content_completed',
                                                   'content_required',
                                                   'checklist_completed',
                                                   'quiz_completed',
                                                   'total_quiz_attempts',
                                                   'discussion_p...
                                                   'total_time_spent_in_content',
                                                   'number_of_logins_to_the_system',
                                                   'age', 'total_course_count',
                                                   'course_count_by_term',
                                                   'completion_ratio',
                                                   'logins_per_course',
                                                   'avg_time_per_login']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['term', 'pseudo_course',
                                                   'gender', 'imm_status'])])),
                ('model', RandomForestClassifier())])

In [250]:
# Predictions
y_pred = full_pipeline.predict(X_test)

In [251]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)

Accuracy: 0.9678111587982833
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      1322
           1       0.97      0.42      0.59        76

    accuracy                           0.97      1398
   macro avg       0.97      0.71      0.79      1398
weighted avg       0.97      0.97      0.96      1398



In [252]:
conf_matrix = confusion_matrix(y_test, y_pred)

print('\nConfusion Matrix:\n', conf_matrix)


Confusion Matrix:
 [[1321    1]
 [  44   32]]
